In [ ]:
import pandas as pd
import plotly.express as px
import argparse
import pathlib

path = pathlib.Path().absolute()
path = str(path).replace("src/drm","")
dossier_graphes=path+"/graphes/"
csv = path+"/data/contrats/export_bi_contrats.csv"  #il manque un ; à la fin du header.
source = "DRM Inter-Rhône"

In [ ]:
#arguments
id_operateur=None

parser = argparse.ArgumentParser()
parser.add_argument("id_operateur", help="Identifiant opérateur", default=id_operateur, nargs='?')

try:
   args = parser.parse_args()
   id_operateur = args.id_operateur
except:
   print("Arguments pas défaut")

In [ ]:
contrats = pd.read_csv(csv, sep=";",encoding="iso8859_15", low_memory=False)
contrats

lastcampagnes = contrats['campagne'].unique()
lastcampagnes.sort()
lastcampagnes = lastcampagnes[-1:]

contrats = contrats.query('campagne in @lastcampagnes')
contrats['couleur'] = contrats['couleur'].str.upper()

contrats.rename(columns = {'identifiant vendeur':'identifiant_vendeur','volume enleve (en hl)':'volume enleve'}, inplace = True)

if(id_operateur):
    contrats = contrats.query("identifiant_vendeur == @id_operateur").reset_index()
    
    
contrats['filtre_produit'] = contrats['appellation'] + "-" + contrats['lieu'] + "-" +contrats['certification']+ "-" +contrats['genre']+ "-" +contrats['mention']

contrats = contrats.groupby(["filtre_produit", "couleur","identifiant acheteur"]).sum(["volume enleve"])[["volume enleve"]]

contrats = contrats.reset_index(level='identifiant acheteur') # same as df.reset_index(level=0)

#contrats

In [ ]:
def create_graphe(df,identifiant,appellation,couleur):
    
    fig = px.pie(df, values='volume enleve', names='identifiant acheteur', color_discrete_sequence=["#e74e62", "#d1342f", "#ac1c25","#961d50","#961d50","#8F1665","#753452","#8A321A"],)

    #fig.show()
    
    dossier = dossier_graphes+"/"+identifiant+"/contrat/"+appellation+"-"+couleur
    pathlib.Path(dossier).mkdir(parents=True, exist_ok=True)

    fig.write_html(dossier+"/graphe1.html",include_plotlyjs=False)
    
    return 

In [ ]:
for bloc in contrats.index.unique():
    df = contrats.loc[bloc]
    create_graphe(df,id_operateur,bloc[0],bloc[1])